In [15]:
import seaborn as sns
import torch
import torch_geometric
import numpy as np
import pandas as pd
from utils import histogram_dicyclic_saver, plot_saved_data, make_path, save_tensors, prediction
import matplotlib.pyplot as plt
from dataset_gen import *
from train import training
import os
import json

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
json_path = 'params_dicyclic.json'
with open(json_path) as f:
        params = json.load(f)

In [16]:
num_reps = params['num_reps']
n_max = params['n_max']
min_num_layers = params['min_num_layers']
max_num_layers = params['max_num_layers']
layers =[i for i in range(min_num_layers, max_num_layers+1)]
epochs = params['epochs']
hd = params['hd']
lr = params['lr']
enc = 'dicyclic'
gnn_type = 'gconv_diff'
gap = params['gap']
early = params['early']
k = params['k']

In [17]:
path = make_path('cycles', gnn_type)
#save config in txt
config_json = pd.read_json(json_path, typ='series')
config_json.to_csv(path+'config.txt')

In [18]:
if not os.path.exists(path):
        os.makedirs(path)
if gap>0:
    data_dict = get_complete_dataset_gap(n_max, gap)
else:
    data_dict = extraction_task_dataset(n_max, k)

In [19]:
for num_l in layers:
    df_data = []
    raw_data = []
    print(f'Number of layers: {num_l}')
    
    for it in range(num_reps):       
        print(str(it+1)+'-th iteration')     
        #seed manual setting
        torch_geometric.seed.seed_everything(it*100)
        #data_dict = dataset_generator(dataset='dicyclic', n_max = n_max, take_random=take_random)
        
        model, raw_data = training(gnn_type, data_dict, 'dicyclic', hd, lr, num_l, epochs, it, raw_data, device, path, early=early)
        data_loader = data_dict['test_loader_nobatch']

        iter_dl = iter(data_loader)
        for i in range(3,n_max+gap+1):
            for j in range(i,n_max+gap+1):
                data = next(iter_dl).to(device)
                y,_ = model(data.x, data.edge_index, data.batch)              
                df_data.append({'1st_cycle':i,'2nd_cycle':j,'rating': float(y), 'it': it+1, 'num_layers':num_l})
                if j!=i:
                    df_data.append({'1st_cycle':j,'2nd_cycle':i,'rating': float(y), 'it': it+1, 'num_layers':num_l})
    df = pd.DataFrame.from_records(df_data)
    df.to_csv(path+'trial_nmax_'+str(n_max-2)+'_gap_'+str(gap+2)+'_'+str(num_l)+'_layers.csv')
    plot_saved_data(path = path, file_name = None, online=False, df=df, num_l = num_l)

Number of layers: 8
1-th iteration
Stopped on Epoch 4 with train accuracy 80.0% and test accuracy 83.33333333333333%
Number of layers: 9
1-th iteration
Stopped on Epoch 3 with train accuracy 100.0% and test accuracy 100.0%


: 